![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Volcano Source Modeling Using InSAR

### Franz J Meyer; University of Alaska Fairbanks 

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /> <font size="3">
    
This notebook will introduce you to the intersection between Radar Remote Sensing and Inverse Modeling. Radar Remote Sensing can provide us with geodetic observations of surface deformation. Inverse Modeling helps us understand the physical causes behind an observed deformation. 
    
To illuminate the handoff from geodesy to geophysics, this notebook will show how to use InSAR observations to determine the most likely parameters of a volcanic magma source underneath Okmok volcano, Alaska. We will use a Mogi source model to describe the physics behind deformation at Okmok. We will again use our **Jupyter Notebook** framework implemented within the Amazon Web Services (AWS) cloud to work on this exercise.

This notebook will introduce the following data analysis concepts:

- A Mogi Source Model describing volcanic source geometry and physics
- How to use the "grid search" method to perform a pseudo-inversion of a Mogi source model 
- How to solve for the best fitting source parameters using modeling with InSAR data

To download, please select the following options in the main menu of the notebook interface:

- **Save your notebook with all of its content** by selecting *File / Save and Checkpoint*
- **To export in Notebook format**, click on *File / Download as / Notebook (.ipynb)*
- **To export in PDF format**, click on *File / Download as / PDF vs LaTeX (.pdf)*

**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

---
## 0. Importing Relevant Python Packages

First step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following scientific libraries:

- [NumPy](http://www.numpy.org/) is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays.
- [Matplotlib](https://matplotlib.org/index.html) is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs.

The first step is to **import all required python modules:**

In [ ]:
%%capture
from pathlib import Path
import copy

%matplotlib inline
import matplotlib.pylab as plt # for add_subplot, cm.jet, colorbar, figure, grid, imshow, rcParams.update, savefig,
                               # set_bad, set_clim, set_title, set_xlabel, set_ylabel
import numpy as np # for arange, arctan, concatenate, cos, fromfile, isnan, ma.masked_value, min, pi, power, reshape,
                   # sqrt, square, sin, sum, tile, transpose, where, zeros     

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

## 1. InSAR at Okmok Volcano, Alaska  

<img style="padding: 7px" src="NotebookAddons/Lab6-OkmokdefoGPS.JPG" width="550" align="right" />

Okmok is one of the more active volcanoes in Alaska’s Aleutian Chain. Its last (known) eruption was in the summer of 2008. Okmok is interesting from an InSAR perspective as it inflates and deflates heavily as magma moves around in its magmatic source located roughly 2.5 km underneath the surface. To learn more about Okmok volcano and its eruptive history, please visit the very informative site of the [Alaska Volcano Observatory](https://avo.alaska.edu/volcanoes/activity.php?volcname=Okmok&eruptionid=604&page=basic).

This notebook uses a pair of C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002 to analyze the properties of a volcanic source that was responsible for an inflation of Okmok volcano of more than 3 cm near its summit. The figure to the right shows the Okmok surface deformation as measured by GPS data from field campaigns conducted in 2000 and 2002. The plots show that the deformation measured at the site is consistent with that created by an inflating point (Mogi) source.

**The primary goal of the problem set is to estimate values for four unknown model parameters describing a source process beneath a volcano.** The notebook uses real InSAR data from Okmok volcano, so you should get some sense for how remote sensing can be used to infer physical processes at volcanoes. We will assume that the source can be modeled as an inflating point source (a so-called Mogi source; see [Lecture 14](https://radar.community.uaf.edu/files/2019/03/2019-Lecture14_UsingInSARinGeophysics.pdf) and will use a grid-search method for finding the source model parameters (3D source location and volume of magma influx) that best describe our InSAR-observed surface deformation.

---
## 2. Download and Plot the Observed Deformation Map
    
### 2.1 Download Data from AWS S3 Storage Bucket

We are using a deformation map created from C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002. The deformation map is **available to you in OpenSARlab's AWS S3 data storage bucket**.

**Create a working directory for this analysis and change into it.**

In [ ]:
path = Path("/home/jovyan/notebooks/SAR_Training/English/Master/data_InSAR_volcano_source_modeling")
if not path.exists():
    path.mkdir()

**Download the deformation map from the AWS-S3 bucket:**

In [ ]:
deformation_map_path = 's3://asf-jupyter-data/E451_20000818_20020719.unw'
deformation_map = str(path/Path(deformation_map_path).name)

!aws --region=us-east-1 --no-sign-request s3 cp $deformation_map_path $deformation_map

**Define some variables:**

In [ ]:
sample = 1100
line = 980
posting = 40.0
half_wave = 28.3

**Read the dataset into the notebook**, storing our observed deformation map in the variable *"observed_deformation_map"*:

In [ ]:
if Path(deformation_map).exists():
    with open (deformation_map, 'rb') as f:    
        coh = np.fromfile(f, dtype='>f', count=-1)
    observed_deformation_map = np.reshape(coh, (line, sample))

**Change the units to cm and replace all nans with 0**

In [ ]:
observed_deformation_map = observed_deformation_map*half_wave/2.0/np.pi
where_are_NaNs = np.isnan(observed_deformation_map)
observed_deformation_map[where_are_NaNs] = 0

**Create a mask** that removes invalid samples (low coherence) from the deformation map:

In [ ]:
observed_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, observed_deformation_map)

---
### 2.2 Visualize The Deformation Map 

We will visualize the deformation map both in units of [cm] and as a rewrapped interferogram.</font>

**Write a function that calculates the bounding box.**

In [ ]:
def extents(vector_component):
        delta = vector_component[1] - vector_component[0]
        return [vector_component[0] - delta/2, vector_component[-1] + delta/2]

**Create a directory in which to store the plots we are about to make, and move into it:**

In [ ]:
product_path = path/'plots'

if not product_path.exists():
    product_path.mkdir()

**Write a plotting function:**

In [ ]:
def plot_model(pth_name, infile, line, sample, posting, output_filename=None, dpi=72):
    # Calculate the bounding box
    extent_xvec = extents((np.arange(1, sample*posting, posting)) / 1000)
    extent_yvec = extents((np.arange(1, line*posting, posting)) / 1000)
    extent_xy = extent_xvec + extent_yvec
    
    plt.rcParams.update({'font.size': 14})
    inwrapped = (infile/10 + np.pi) % (2*np.pi) - np.pi
    cmap = copy.copy(plt.cm.get_cmap("jet"))
    cmap.set_bad('white', 1.)
    
    # Plot displacement
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(1, 2, 1)
    im = ax1.imshow(infile, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
    ax1.set_title("Displacement in look direction [mm]")
    ax1.set_xlabel("Easting [km]")
    ax1.set_ylabel("Northing [km]")
    plt.grid()
    
    # Plot interferogram
    im.set_clim(-30, 30)
    ax2 = fig.add_subplot(1, 2, 2)
    im = ax2.imshow(inwrapped, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax2.figure.colorbar(im, ax=ax2, orientation='horizontal')
    ax2.set_title("Interferogram phase [rad]")
    ax2.set_xlabel("Easting [km]")
    ax2.set_ylabel("Northing [km]")
    plt.grid()
    
    if output_filename:
        plt.savefig(f'{pth_name}/{output_filename}', dpi=dpi)

Call plot_model() to **plot our observed deformation map:**

In [ ]:
plot_model(product_path, observed_deformation_map_m, line, sample, posting, output_filename='Okmok-inflation-observation.png', dpi=200)

---
## 3. The Mogi Source Model and InSAR
    
### 3.1 The Mogi Equations
The Mogi model provides the 3D ground displacement, $u(x,y,z)$, due to an inflating source at location $(x_s,y_s,z_s)$ with volume change $V$:

\begin{equation}
u(x,y,z)=\frac{1}{\pi}(1-\nu)\cdot V\Big(\frac{x-x_s}{r(x,y,z)^3},\frac{y-y_s}{r(x,y,z)^3},\frac{z-z_s}{r(x,y,z)^3}\Big)
\end{equation}

\begin{equation}
r(x,y,z)=\sqrt{(x-x_s)^2+(y-y_s)^2+(z-z_s)^2}
\end{equation}

where $r$ is the distance from the Mogi source to $(x,y,z)$, and $\nu$ is the Poisson's ratio of the halfspace. The Poisson ratio describes how rocks react when put under stress (e.g., pressure). It is affected by temperature, the quantity of liquid to solid, and the composition of the soil material. **In our problem, we will assume that $\nu$ is fixed**.

---
### 3.2 Projecting Mogi Deformation to InSAR Line-of-Sight

In our example, the $x$-axis points east, $y$ points north, and $z$ points up. However, in the code the input values for $z$ are assumed to be depth, such that the Mogi source is at depth $z_s > 0$. The observed interferogram is already corrected for the effect of topography, so the observations can be considered to be at $z = 0$.
    
<img style="padding: 7px" src="NotebookAddons/Lab6-LOSprojection.JPG" width="650" align="center" />
The satellite “sees” a projection of the 3D ground displacement, $u$, onto the look vector, $\hat{L}$, which points from the satellite to the target. Therefore, we are actually interested in the (signed magnitude of the) projection of $u$ onto $\hat{L}$ (right). This is given by

\begin{array}{lcl} proj_{\hat{L}}u & = & (u^T\hat{L})\hat{L} \\ u^T\hat{L} & = & u \cdot \hat{L} = |u||\hat{L}|cos(\alpha) = |u|cos(\alpha) \\ & = & u_x\hat{L}_x+ u_y\hat{L}_y + u_z\hat{L}_z \end{array}

where the look vector is given by $\hat{L}=(sin(l) \cdot cos(t), -sin(l) \cdot sin(t), -cos(l))$, where $l$ is the look angle measured from the nadir direction and $t$ is the satellite track angle measured clockwise from geographic north. All vectors are represented in an east-north-up basis.

Our forward model takes a Mogi source, $(x_s,y_s,z_s,V)$, and computes the look displacement at any given $(x, y, z)$ point. If we represent the <i>i</i>th point on our surface grid by $x_i = (x_i,y_i,z_i)$ the the displacement vector is $u_i = u(x_i, y_i, z_i)$, and the look displacement is

\begin{equation}
d_i = u_i \cdot \hat{L}
\end{equation}

---
### 3.3 Defining the Mogi Forward Model

We can now represent the Mogi *forward problem* as 

\begin{equation}
g(m) = d
\end{equation}

where $g(·)$ describes the forward model in the very first equation in this notebook, $m$ is the (unknown) Mogi model, and $d$ is the predicted interferogram. The following code cells calculate the Mogi forward model according to the equations given above:

**Write a function to calculate a forward model for a Mogi source:**

In [ ]:
def calc_forward_model_mogi(n1, e1, depth, delta_volume, northing, easting, plook):
    
    # This geophysical coefficient is needed to describe how pressure relates to volume change
    displacement_coefficient = (1e6*delta_volume*3)/(np.pi*4)
    
    # Calculating the horizontal distance from every point in the deformation map to the x/y source location
    d_mat = np.sqrt(np.square(northing-n1) + np.square(easting-e1))
    
    # denominator of displacement field for mogi source
    tmp_hyp = np.power(np.square(d_mat) + np.square(depth),1.5)
    
    # horizontal displacement
    horizontal_displacement = displacement_coefficient * d_mat / tmp_hyp
    
    # vertical displacement
    vertical_displacement = displacement_coefficient * depth / tmp_hyp
    
    # azimuthal angle
    azimuth = np.arctan2((easting-e1), (northing-n1))
    
    # compute north and east displacement from horizontal displacement and azimuth angle
    east_displacement = np.sin(azimuth) * horizontal_displacement
    north_displacement = np.cos(azimuth) * horizontal_displacement
    
    # project displacement field onto look vector
    temp = np.concatenate((east_displacement, north_displacement, vertical_displacement), axis=1)
    delta_range = temp.dot(np.transpose([plook]))
    delta_range = -1.0 * delta_range
    return delta_range

**Write a function to create simulated deformation data based on Mogi Source Model parameters:**

In [ ]:
def deformation_data_from_mogi(x, y, z, volume, iplot, imask):
    # Organizing model parameters
    bvc = [x, y, z, volume, 0, 0, 0, 0]
    bvc = np.asarray(bvc, dtype=object)
    bvc = np.transpose(bvc)
    
    # Setting acquisition parameters
    track =  -13.3*np.pi / 180.0
    look  = 23.0*np.pi / 180.0
    plook = [-np.sin(look)*np.cos(track), np.sin(look)*np.sin(track), np.cos(look)]
    
    # Defining easting and northing vectors
    northing = np.arange(0, (line)*posting, posting) / 1000
    easting = np.arange(0, (sample)*posting, posting) / 1000
    northing_mat = np.tile(northing, (sample, 1))
    easting_mat = np.transpose(np.tile(easting, (line, 1)))
    northing_vec = np.reshape(northing_mat, (line*sample, 1))
    easting_vec = np.reshape(easting_mat, (line*sample, 1))
    
    # Handing coordinates and model parameters over to the rngchg_mogi function
    calc_range = calc_forward_model_mogi(bvc[1], bvc[0], bvc[2], bvc[3], northing_vec, easting_vec, plook)
    
    # Reshaping surface deformation data derived via calc_forward_model_mogi()
    surface_deformation = np.reshape(calc_range, (sample,line))
    
    # return rotated surface deformation
    return np.transpose(np.fliplr(surface_deformation))

---
### 3.4 Plotting The Mogi Forward Model

The cell below plots several Mogi forward models by varying some of the four main Mogi modeling parameters $(x_s,y_s,z_s,V)$.
    
The examples below fix the `depth` parameter to $z_s = 2.58 km$ and the `volume` change parameter to $volume = 0.0034 km^3$. We then vary the `easting` and `northing` parameters $x_s$ and $y_s$ to demonstrate how the model predictions vary when model parameters are changed.

**Run the first example:**

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, sample*posting, posting))/1000)
extent_y = extents((np.arange(1, line*posting, posting))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

zs = 2.58;
volume = 0.0034;
xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = copy.copy(plt.cm.get_cmap("jet"))
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        im = ax.imshow(predicted_deformation_map_m, cmap=cmap, extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title('Source: X=%4.2fkm; Y=%4.2fkm' % (xs[k], ys[l]))
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1
        
plt.savefig(f'{product_path}/Model-samples-3by3.png', dpi=200, transparent='false')

---
## 4. Solving the [Pseudo]-Inverse Model

The inverse problem seeks to determine the optimal parameters $(\hat{x_s},\hat{y_s},\hat{z_s},\hat{V})$ of the Mogi model $m$ by minimizing the <i>misfit</i> between predictions, $g(m)$, and observations $d^{obs}$ according to
    
\begin{equation}
\sum{\Big[g(m) - d^{obs}\Big]^2}
\end{equation}

This equation describes misfit using the *method of least-squares*, a standard approach to approximate the solution of an overdetermined equation system. We will use a *grid-search* approach to find the set of model parameters that minimize the the misfit function. 

**The approach is composed of the following processing steps:**

1. Loop through the mogi model parameters
1. Calculate the forward model for each set of parameters
1. Calculate the misfit $\sum{[g(m) - d^{obs}]^2}$
1. Find the parameter set that minimizes this misfit

---

### 4.1 Experimenting with Misfit

**Let's look at the misfit $\sum{[g(m) - d^{obs}]^2}$ for a number of different model parameter sets $(x_s,y_s,z_s,V)$:**

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, sample*posting, posting))/1000)
extent_y = extents((np.arange(1, line*posting, posting))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

zs = 2.58;
volume = 0.0034;
xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = copy.copy(plt.cm.get_cmap("jet"))
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        im = ax.imshow(observed_deformation_map_m-predicted_deformation_map_m, cmap=cmap, extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k], ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title('Source: X=%4.2fkm; Y=%4.2fkm' % (xs[k], ys[l]))
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1
plt.savefig(f'{product_path}/Misfit-samples-3by3.png', dpi=200, transparent='false')

---
### 4.2 Running Grid-Search to Find Best Fitting Model Parameters $(\hat{x}_s,\hat{y}_s)$

The following code cell runs a grid-search approach to find the best fitting Mogi source parameters for the 2000-2002 deformation event at Okmok. To keep things simple, we will fix the depth $z_s$ and volume change $V$ parameters close to their "true" values and search only for the correct east/north source location ($x_s,y_s$).

**Write a script using the grid-search approach in Python:**

In [ ]:
# FIX Z AND dV, SEARCH OVER X AND Y

# Setting up search parameters
xs = np.arange(19, 22.2, 0.2)
ys = np.arange(21, 23.2, 0.2)
zs = 2.58;
volume = 0.0034;

nx = xs.size
ny = ys.size
ng = nx * ny;

print(f"fixed z = {zs}km, dV = {volume}, searching over (x,y)")

misfit = np.zeros((nx, ny))
subplot_index = 0

# Commence grid-search for best model parameters
for k, xv in enumerate(xs):
    for l, yv in enumerate(ys):
        subplot_index += 1
        predicted_deformation_map = deformation_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)
        misfit[k,l] = np.sum(np.square(observed_deformation_map_m - predicted_deformation_map_m))
    print(f"Source {subplot_index:3d}/{ng:3d} is x = {xs[k]:.2f} km, y = {ys[l]:.2f} km")

# Searching for the minimum in the misfit matrix
mmf = np.where(misfit == np.min(misfit))
print(f"\n----------------------------------------------------------------")
print('Best fitting Mogi Source located at: X = %5.2f km; Y = %5.2f km' % (xs[mmf[0]], ys[mmf[1]]))
print(f"----------------------------------------------------------------")

---
### 4.3 Plot and Inspect the Misfit Function

The code cell below plots the misfit function ($\sum{[g(m) - d^{obs}]^2}$) describing the fit of different Mogi source parameterizations to the observed InSAR data. You should notice a clear minimum in the misfit plot at the location of the best fitting source location estimated above. 
    
You may notice that, even for the best fitting solution, the misfit does not become zero. This could be due to other signals in the InSAR data (e.g., atmospheric effects or residual topography). Alternatively, it could also indicate that the observed deformation doesn't fully comply with Mogi theory. 

**Plot the misfit function ($\sum{[g(m) - d^{obs}]^2}$):**

In [ ]:
plt.rcParams.update({'font.size': 18})
extent_xy = extents(xs) + extents(ys)
fig = plt.figure(figsize=(10, 10))
cmap = copy.copy(plt.cm.get_cmap("jet"))
ax1 = fig.add_subplot(1, 1 ,1)
im = ax1.imshow(np.transpose(misfit), origin='lower', cmap=cmap, extent=extent_xy)
# USE THIS COMMAND TO CHANGE COLOR SCALING: im.set_clim(-30, 30)
ax1.set_aspect('auto')
cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
ax1.plot(xs[mmf[0]], ys[mmf[1]], 'k*', markersize=25, markerfacecolor='w')
ax1.set_title("Misfit Function for Mogi-Source Approximation")
ax1.set_xlabel("Easting [km]")
ax1.set_ylabel("Northing [km]")
plt.savefig(f'{product_path}/Misfit-function.png', dpi=200, transparent='false')

---
### 4.4 Plot Best-Fitting Mogi Forward Model and Compare to Observations

With the best-fitting model parameters defined, you can now analyze how well the model fits the InSAR-observed surface deformation. The best way to do that is to look at both the observed and predicted deformation maps and compare their spatial patterns. Additionally, we will also plot the residuals (*observed_deformation_map* - *observed_deformation_map*) to determine if there are additional signals in the data that are not modeled using Mogi theory. 

**Compare the observed and predicted deformation maps:**

In [ ]:
# Calculate predicted deformation map for best-fitting Mogi parameters:
predicted_deformation_map = deformation_data_from_mogi(xs[mmf[0]], ys[mmf[1]], zs, volume, 0, 0)

# Mask the predicted deformation map to remove pixels incoherent in the observations:
predicted_deformation_map_m = np.ma.masked_where(observed_deformation_map==0, predicted_deformation_map)

# Plot observed deformation map
plot_model(product_path, observed_deformation_map_m, line, sample, posting)

# Plot simulated deformation map
plot_model(product_path, predicted_deformation_map_m, line, sample, posting)

plt.savefig(f'{product_path}/BestFittingMogiDefo.png', dpi=200, transparent='false')

# Plot simulated deformation map without mask applied
plot_model(product_path, predicted_deformation_map, line, sample, posting)

**Determine if there are additional signals in the data that are not modeled using Mogi theory:**

In [ ]:
# Plot residual between observed and predicted deformation maps
plot_model(product_path, observed_deformation_map_m-predicted_deformation_map_m, line, sample, posting)
plt.savefig(f'{product_path}/Residuals-ObsMinusMogi.png', dpi=200, transparent='false')

*InSAR_volcano_source_modeling.ipynb - Version 1.3.2 - November 2021*

*Version Changes*

- *asf_notebook -> opensarlab_lib*
- *html -> markdown*
- *url-widget*